In [ ]:
# import libraries
import pandas as pd
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# project specific libs
import os
import matplotlib.pyplot as plt
import pathlib

In [ ]:
# ALBA UPGMA and KMEANS
from scipy.spatial import distance
from scipy.cluster import hierarchy
from sklearn.metrics import v_measure_score, silhouette_score
from scipy.cluster.vq import whiten

# load in alba df after converting to tsv
df_bc = pd.read_csv(path + 'outputs/jobs15/distance-matrix.tsv', sep='\t', index_col=0)
samples = df_bc.index.values
X = df_bc.values

# UPGMA
# this function magically works both ways!
# get condensed distance matrix
# condensed_dist = distance.pdist(data, metric='euclidean')
condensed_dist = distance.squareform(X)

# construct linkages
linkages = hierarchy.linkage(condensed_dist, method='average', metric='euclidian')

# kmeans raw data
df_asv = pd.read_csv(path + 'outputs/jobs15/no-clades-table-I7-spp.tsv', sep='\t', index_col=0).T
# normalize by row total
df_asv = df_asv.div(df_asv.sum(axis=1), axis=0)
X = df_asv.values

# whiten data (scaling)
features = whiten(X)

data = {}
data['samples'] = samples

for k in [2,3]:
    # get cut for k clusters
    cut = hierarchy.fcluster(linkages, k, criterion='maxclust')
    ss = silhouette_score(X, cut, metric='euclidean')
    print(ss)
    col = 'UPGMA_' + str(k)
    data[col] = cut
    
    centroid, label = scipy.cluster.vq.kmeans2(features, iter=100, k=k)#, iter=20, thresh=1e-05, check_finite=True, *, seed=None)
    ss = silhouette_score(X, label, metric='euclidean')
    print(ss)

    col = 'kmeans_' + str(k) 
    data[col] = label

df = pd.DataFrame.from_dict(data)
df = df.set_index('samples')
df.to_csv(path + 'outputs/jobs15/UPGMA_kmeans_2_3.tsv', sep='\t')
df